In [39]:
import numpy as np

from nltk.metrics.distance import edit_distance

ADD_COST = 1
DEL_COST = 1
REPL_COST = 1


def levenshtein(s1, s2):
    n1 = len(s1) + 1
    n2 = len(s2) + 1
    D = np.zeros((n1, n2))
    D[:, 0] = np.arange(n1)
    D[0, :] = np.arange(n2)

    for r in range(1, n1):
        for c in range(1, n2):
            cost_add = D[r, c - 1] + ADD_COST
            cost_del = D[r - 1, c] + DEL_COST
            if s1[r - 1] == s2[c - 1]:
                cost_sub = D[r - 1, c - 1]
            else:
                cost_sub = D[r - 1, c - 1] + REPL_COST
            D[r, c] = min([cost_add, cost_del, cost_sub])
    return D[n1 - 1, n2 - 1]


def levenshtein_2(s1, s2):
    n1 = len(s1) + 1
    n2 = len(s2) + 1
    previous = np.arange(n2)
    current = np.zeros(n2)
    for r in range(1, n1):
        current[0] = r
        for c in range(1, n2):
            cost_add = current[c - 1] + ADD_COST
            cost_del = previous[c] + DEL_COST
            if s1[r - 1] == s2[c - 1]:
                cost_sub = previous[c - 1]
            else:
                cost_sub = previous[c - 1] + REPL_COST
            current[c] = min([cost_add, cost_del, cost_sub])
        previous, current = current, previous
    return previous[-1]


s1 = 'to be or not to be'
s2 = 'doo be doo be doo'

print(levenshtein(s1, s2))
print(levenshtein_2(s1, s2))
print(edit_distance(s1, s2, substitution_cost=REPL_COST))

11.0
11
11


['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [55]:
x = 'vacation'
x[8:]

''

In [102]:
LOWERCASE = [chr(x) for x in range(ord('a'), ord('z') + 1)]
UPPERCASE = [chr(x) for x in range(ord('A'), ord('Z') + 1)]
LOWERCASE_OTHERS = ['ç', 'á', 'â', ]  # etc.
UPPERCASE_OTHERS = [x.upper() for x in LOWERCASE_OTHERS]
LETTERS = LOWERCASE + UPPERCASE + LOWERCASE_OTHERS + UPPERCASE_OTHERS

def edit1(text):
    words = []
    
    # Fase 1: as remoçoes.
    for p in range(len(text)):
        new_word = text[:p] + text[p + 1:]
        if len(new_word) > 0:
            words.append(new_word)
        
    # Fase 2: as adições.
    for p in range(len(text) + 1):
        for c in LETTERS:
            new_word = text[:p] + c + text[p:]
            words.append(new_word)
    
    # Fase 3: as substituições.
    for p in range(len(text)):
        orig_c = text[p]
        for c in LETTERS:
            if orig_c != c:
                new_word = text[:p] + c + text[p + 1:]
                words.append(new_word)
    
    return set(words)

def edit2(text):
    words1 = edit1(text)
    words2 = set()
    for w in words1:
        candidate_words2 = edit1(w)
        candidate_words2 -= words1
        words2.update(candidate_words2)
    words2 -= set([text])
    return words2


In [75]:
import json

from pathlib import Path


def read_data(filename):
    with open(filename, 'r') as file:
        data = [json.loads(line) for line in file]
    return data

In [76]:
data = read_data(Path('..', 'Aula04', 'dump_small_clean.jsonln'))
# data = read_data(Path('..', 'Aula04', 'dump_small.jsonln'))

In [103]:
import re


def minusculas(tokens):
    return [token.lower() for token in tokens]

def remove_digitos(tokens):
    return [token for token in tokens if re.fullmatch('[^\d]*', token)]

def pega_palavras(tokens):
    return [token for token in tokens if re.fullmatch('\w+', token)]
    
def limpa_tokens(tokens):
    #tokens = minusculas(tokens)
    tokens = remove_digitos(tokens)
    tokens = pega_palavras(tokens)    
    return tokens

In [104]:
from nltk import word_tokenize
from tqdm import tqdm

all_words = []
for item in tqdm(data):
    texto = item['body']
    tokens = word_tokenize(texto)
    tokens = limpa_tokens(tokens)
    all_words += tokens

100%|███████████████████████████████████████████████████████████████████████████| 11225/11225 [00:59<00:00, 188.85it/s]


In [105]:
from collections import Counter
word_counts = Counter(all_words)

In [106]:
word_counts_list = list(word_counts.items())

In [107]:
word_counts_list_sorted = sorted(word_counts_list, key=lambda x: (-x[1], x[0]))

In [156]:
vocab = word_counts_list_sorted[:10000]

In [157]:
vocab = dict(vocab)

In [160]:
word = 'pars'
if word in vocab:
    candidatos = [word]
else:
    candidatos = []
candidatos += \
    [w for w in edit1(word) if w in vocab] \
    + [w for w in edit2(word) if w in vocab] \
    + [word]

In [161]:
candidatos

['part',
 'pais',
 'Wars',
 'pares',
 'Mars',
 'para',
 'par',
 'per',
 'bar',
 'ar',
 'raras',
 'War',
 'page',
 'pai',
 'Faro',
 'Nara',
 'dar',
 'por',
 'Hans',
 'Mas',
 'paz',
 'Das',
 'Stars',
 'passo',
 'Gary',
 'Haas',
 'Park',
 'pra',
 'pano',
 'Was',
 'padre',
 'Mar',
 'Par',
 'Cara',
 'Lara',
 'caro',
 'mas',
 'papa',
 'passe',
 'Days',
 'Las',
 'Mark',
 'hard',
 'mares',
 'pts',
 'Pará',
 'Nas',
 'Pais',
 'Marc',
 'Mans',
 'Carl',
 'was',
 'Barks',
 'as',
 'Bass',
 'Part',
 'Hard',
 'pro',
 'mais',
 'pois',
 'parar',
 'padres',
 'nas',
 'lar',
 'passa',
 'Mais',
 'Bar',
 'art',
 'cara',
 'Far',
 'Tais',
 'tais',
 'Dark',
 'aos',
 'Sara',
 'Mary',
 'parte',
 'var',
 'Karl',
 'partes',
 'parou',
 'mar',
 'Para',
 'las',
 'raro',
 'das',
 'are',
 'persa',
 'Paris',
 'pars']

In [126]:
vocab

{'de': 220263,
 'e': 88145,
 'a': 78953,
 'do': 62915,
 'o': 60541,
 'da': 60083,
 'em': 55634,
 'que': 41304,
 'um': 29584,
 'com': 28200,
 'uma': 27841,
 'no': 27733,
 'Categoria': 27579,
 'para': 25973,
 'na': 24444,
 'é': 23769,
 'por': 21161,
 'ref': 20209,
 'foi': 19726,
 'como': 17780,
 'os': 17021,
 'dos': 16781,
 'A': 16049,
 'O': 15547,
 'ao': 11423,
 'as': 11288,
 'br': 11276,
 'se': 11104,
 'center': 10942,
 'sua': 10455,
 'mais': 9878,
 'Em': 9359,
 'das': 8936,
 'seu': 8744,
 'não': 8609,
 'à': 7735,
 'também': 7427,
 'ou': 7360,
 'pela': 7153,
 'pelo': 6555,
 'ser': 5588,
 'of': 5543,
 'small': 5499,
 'entre': 5451,
 'The': 4901,
 'São': 4832,
 'são': 4832,
 'anos': 4595,
 'nos': 4523,
 'era': 4445,
 'nbsp': 4388,
 'Brasil': 4269,
 'left': 4263,
 'ele': 4260,
 'the': 4048,
 'No': 4031,
 'foram': 4004,
 'mas': 3981,
 'sobre': 3963,
 'Os': 3927,
 'cidade': 3803,
 'seus': 3592,
 'até': 3579,
 'onde': 3392,
 'Rio': 3261,
 'nas': 3244,
 'Estados': 3154,
 'Ligações': 3027,
 'e